# MaxDome Notebooks

MaxDome notebooks are Serverless PySpark notebooks powered by AWS Glue Interactive Sessions.
To Create a session and begin using PySpark simply run a cell of code. 

## Additional Sample Notebooks

Additonal sample notebooks are available on the aws-glue-sample repo in GitHub and can be imported to your project using the upload button on the top of the file browser in Jupyter.

https://github.com/aws-samples/aws-glue-samples/tree/master/examples/notebooks


#### Optional: Configuration

MaxDome notebooks are configured via Jupyter magics (commands prefixed with `%` and `%%`). In MaxDome and Glue IS, these are used to configure the PySpark environment including cluster size, shape and installed libraries. 

run `%help` in an empty cell to see the full list of magic commands.

In [ ]:
%help

In [ ]:
# Start Session and configure Spark
import sys
import boto3
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import col


args = getResolvedOptions(
    sys.argv, ["redshift_url", "redshift_iam_role", "redshift_tempdir"]
)

# These exist in Sessions automatically. Adding for Linter
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

In [ ]:
# Get the MaxDome Database info
glue_client = boto3.client("glue")

# filter Glue Databases for databases that start with "maxdome_producer_db"
databases_paginator = glue_client.get_paginator("get_databases")
response_iterator = databases_paginator.paginate()
glue_databases = response_iterator.build_full_result().get("DatabaseList")
maxdome_database = [
    db for db in glue_databases if db["Name"].startswith("maxdome_producer_db")
][0]

maxdome_database_name = maxdome_database["Name"]
maxdome_database_location = maxdome_database["LocationUri"]
print(f"Project Database Name: {maxdome_database_name}")
print(f"Project Database Location: {maxdome_database_location}")

## Create a Spark DataFrame from Titanic Dataset


In [ ]:
df = spark.read.csv(f"s3://sagemaker-example-files-prod-{boto3.session.Session().region_name}/datasets/tabular/dirty-titanic/", header=True)
df.show(5, truncate=False)
df.printSchema()

In [ ]:
from pyspark.sql.types import StringType, IntegerType

types_corrected = df.select(
    col("index").cast("integer"),
    col("survived").cast("boolean"),
    col("name").cast("string"),
    col("sex").cast("string"),
    col("age").cast("integer"),
    col("sibsp").cast("float"),
    col("parch").cast("float"),
    col("ticket").cast("integer"),
    col("fare").cast("double"),
    col("cabin").cast("string"),
    col("embarked").cast("string"),
    col("boat").cast("integer"),
    col("`home.dest`").cast("string").alias("home_and_dest"),
    col("month_of_departure").cast("string"),
)
types_corrected.show(5)

## Clean the data using the pandas API on Spark


For those new to or unfamiliar with PySpark the Pandas API on Spark may allow for a more familiar interface. Using the library much of the operations on a Spark DataFrame can be executed using familiar pandas APIs while taking advantage of many of Spark's optimizations and distributed compute. 

See the [Spark documentation](https://spark.apache.org/docs/latest/api/python/getting_started/quickstart_ps.html) for a deeper dive

In [ ]:
# enable the pandas API on the Spark Dataframe
ps_df = types_corrected.pandas_api()

# Fill missing age
ps_df["age"] = ps_df["age"].fillna(ps_df["age"].astype(int).median())

# Drop null survived
ps_df = ps_df[ps_df["survived"].notnull()]

# Split Home and dest into two columns
ps_df[["home", "dest"]] = ps_df["home_and_dest"].str.split("/", n=1, expand=True)
ps_df = ps_df.drop(["home_and_dest"], axis=1)


# Convert back to a Spark Dataframe for write
spark_df = ps_df.to_spark()
spark_df.show()

## Writing the dataframe to a catalog table



#### Writing to the data catalog

In [ ]:
DATABASE = maxdome_database_name
TABLE = "titanic_clean"
S3_PATH = f"{maxdome_database_location}{TABLE}/"
(spark_df.write.mode("append").option("path", S3_PATH).saveAsTable(f"{DATABASE}.{TABLE}"))

#### Reading the data back from Catalog

In [ ]:
df2 = spark.sql(f"select * from {DATABASE}.{TABLE}")
df2.show()

## Redshift Operations

Redshift connectivity is currently handled by the community Spark Connector for Redshift and is best suited for moving large amounts of data between Redshift and Spark for use with PySpark or pandas.
> A library to load data into Spark SQL DataFrames from Amazon Redshift, and write them back to Redshift tables. Amazon S3 is used to efficiently transfer data in and out of Redshift, and JDBC is used to automatically trigger the appropriate COPY and UNLOAD commands on Redshift.
>
> This library is more suited to ETL than interactive queries, since large amounts of data could be extracted to S3 for each query execution. If you plan to perform many queries against the same Redshift tables then we recommend saving the extracted data in a format such as Parquet.

Parameters and documentation can be found [on GitHub](https://github.com/spark-redshift-community/spark-redshift)

#### Write to Redshift with IAM

In [ ]:
rs_database = "dev"
rs_table = "project.titanic"

(
    spark_df.write.format("io.github.spark_redshift_community.spark.redshift")
    .option("url", args["redshift_url"])
    .option("dbtable", rs_table)
    .option("tempdir", args["redshift_tempdir"])
    .option("aws_iam_role", args["redshift_iam_role"])
    .mode("overwrite")
    .save()
)

#### Read from Redshift

In [ ]:
rs_read_df = (
    spark.read.format("io.github.spark_redshift_community.spark.redshift")
    .option("url", args["redshift_url"])
    .option("aws_iam_role", args["redshift_iam_role"])
    .option("tempdir", args["redshift_tempdir"])
    .option("unload_s3_format", "PARQUET")
    .option("dbtable", rs_table)
    .load()
)
rs_read_df.show(5)

## Installing additional Python modules

To install additional Python modules, such as the SageMaker PySDK, run `%additional_python_modules` at the start of your session. Note that to install Python modules without S3, you'll need internet access from your MaxDome VPC. For more info, please refer to https://docs.aws.amazon.com/glue/latest/dg/manage-notebook-sessions.html#specify-default-modules

In [ ]:
%stop_session